In [ ]:
import os 

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")

# Function to create model-specific configuration
def get_model_config(model_name):

    api_key = azure_openai_api_key
    
    if model_name == "o4-mini":
        return {
            "azure_endpoint": azure_openai_endpoint,
            "azure_deployment": "o4-mini",
            "api_key": api_key,
            "api_version": "2025-01-01-preview",
        }
    elif model_name == "gpt5":
        return {
            "azure_endpoint": azure_openai_endpoint,
            "azure_deployment": "gpt-5",
            "api_key": api_key,
            "api_version": "2025-01-01-preview",
        }
    elif model_name == "ministral":
        return {
            "azure_endpoint": azure_openai_endpoint,
            "azure_deployment": "Ministral-3B",
            "api_key": api_key,
            "api_version": "2024-05-01-preview",
        }
    else:
        # Default configuration from environment variables
        api_version = azure_openai_api_version
        if api_version and api_version < "2024-12-01-preview":
            api_version = "2024-12-01-preview"
        
        return {
            "azure_endpoint": azure_openai_endpoint,
            "azure_deployment": azure_openai_deployment,
            "api_key": azure_openai_api_key,
            "api_version": api_version,
        }

In [ ]:
import os 

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_GPT5_PRO")
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT_GPT5_PRO")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY_GPT5_PRO")  
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION_GPT5_PRO")

model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

In [ ]:
import pathlib
import random

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    QAEvaluator,
    #F1ScoreEvaluator,
    FluencyEvaluator,
)
from model_endpoints import ModelEndpoints

models = [
    "o4-mini",
    "gpt5",
    "ministral",
]

# Define a simple answer length evaluator
def answer_length(response, **kwargs):
    return {"answer_length": len(response)}

path = str(pathlib.Path(pathlib.Path.cwd())) + "/data/software_engineering_data.jsonl"

for model in models:
    # Get model-specific configuration
    model_config = get_model_config(model)
    
    randomNum = random.randint(1111, 9999)
    
    # Create evaluators with model-specific configuration
    relevance_evaluator = RelevanceEvaluator(model_config)
    coherence_evaluator = CoherenceEvaluator(model_config)
    groundedness_eval = GroundednessEvaluator(model_config=model_config)
    #f1_eval = F1ScoreEvaluator(threshold=0.6)
    qa_eval = QAEvaluator(model_config=model_config)
    fluency_eval = FluencyEvaluator(model_config=model_config)
    
    print(f"Running evaluation for model: {model}")
    evaluation_name = "Eval-Run-" + str(randomNum) + "-" + model.title()
    results = evaluate(
        evaluation_name=evaluation_name,
        data=path,
        target=ModelEndpoints(model),
        evaluators={
            #"groundedness": groundedness_eval,
            "answer_length": answer_length,
            "qa": qa_eval,
            #"f1-score": f1_eval,
            #"fluency": fluency_eval
        },
        azure_ai_project=azure_ai_project,
        evaluator_config={
            "relevance": {
                "column_mapping": {
                    "response": "${target.response}",
                    "context": "${data.context}",
                    "query": "${data.query}",
                },
            },
        },
        output_path="./software_engineering-" + evaluation_name + ".json"
    )